# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [24]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from io import StringIO
import time

# Configurar Selenium y ChromeDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Opcional: Ejecutar el navegador en modo headless

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# Descargar HTML con Selenium
url = "https://companies-market-cap-copy.vercel.app/index.html"
driver.get(url)
time.sleep(3)  # Esperar a que la página se cargue completamente

html_content = driver.page_source

# Cerrar el navegador
driver.quit()

# Transformar el HTML
soup = BeautifulSoup(html_content, 'html.parser')
tables = soup.find_all('table')

# Suponiendo que la tabla con la evolución anual es la primera tabla
annual_growth_table = tables[0]
df = pd.read_html(StringIO(str(annual_growth_table)))[0]

# Procesar el DataFrame
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.replace('$', '').str.replace('B', '').str.strip() if x.dtype == "object" else x)
df.dropna(inplace=True)

# Convertir columnas a numéricas
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')
df['Change'] = df['Change'].str.replace('%', '').astype(float)
df.dropna(subset=['Year', 'Revenue', 'Change'], inplace=True)

# Verificar nombres de columnas y primeros registros
print(df.columns)  # Esto debería imprimir: Index(['Year', 'Revenue', 'Change'], dtype='object')
print(df.head())

# Almacenar los datos en sqlite
# Conectar a la base de datos (o crearla si no existe)
conn = sqlite3.connect('company_growth.db')

# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()

# Crear la tabla 'annual_growth'
cursor.execute('''
CREATE TABLE IF NOT EXISTS annual_growth (
    Year INTEGER,
    Revenue REAL,
    Change REAL
)
''')

# Insertar los valores del DataFrame en la tabla
for index, row in df.iterrows():
    cursor.execute('''
    INSERT INTO annual_growth (Year, Revenue, Change)
    VALUES (?, ?, ?)
    ''', (row['Year'], row['Revenue'], row['Change']))

# Guardar (commit) los cambios
conn.commit()

# Cerrar la conexión
conn.close()

# Visualizar los datos
# Gráfico de líneas

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5949847194e3 <unknown>
#1 0x594984448c76 <unknown>
#2 0x59498446f757 <unknown>
#3 0x59498446e029 <unknown>
#4 0x5949844acccc <unknown>
#5 0x5949844ac47f <unknown>
#6 0x5949844a3de3 <unknown>
#7 0x5949844792dd <unknown>
#8 0x59498447a34e <unknown>
#9 0x5949846d93e4 <unknown>
#10 0x5949846dd3d7 <unknown>
#11 0x5949846e7b20 <unknown>
#12 0x5949846de023 <unknown>
#13 0x5949846ac1aa <unknown>
#14 0x5949847026b8 <unknown>
#15 0x594984702847 <unknown>
#16 0x594984712243 <unknown>
#17 0x737ca01faea7 start_thread
